<a href="https://colab.research.google.com/github/namrataawagh/MachineLearning/blob/main/Binning%26Binarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binning and Binarization:
Both binning and binarization are data preprocessing techniques used to transform continuous variables into categorical ones,

## Binning (Discretization)

Binning divides continuous data into multiple discrete intervals (bins)

Why use discretization?
1. Handle outliers
2. To improve the values spread


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer

from sklearn.compose import ColumnTransformer

In [ ]:
df = pd.read_csv('train.csv',usecols=['Age','Fare','Survived'])

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape


(714, 3)

In [ ]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [ ]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train.head()

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542
253,30.0,16.1000
719,33.0,7.7750
666,25.0,13.0000


In [ ]:
clf = DecisionTreeClassifier()

In [ ]:

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_pred)


0.6433566433566433

In [ ]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring ='precision'))

np.float64(0.5507411038310869)

In [ ]:
kbin_age = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')
kbin_fare = KBinsDiscretizer(n_bins =15, encode='ordinal',strategy = 'quantile')

In [ ]:
trf = ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1])
])

In [ ]:
X_train_t = trf.fit_transform(X_train)
X_test_t = trf.transform(X_test)

In [ ]:
trf.named_transformers_['first'].n_bins_   # no. of bins created

array([15])

In [ ]:
trf.named_transformers_['first'].bin_edges_  # intervals of 10% then 20% incrementing quantiles

array([array([ 0.42,  6.  , 16.  , 19.  , 21.  , 23.  , 25.  , 28.  , 30.  ,
              32.  , 35.  , 38.  , 42.  , 47.  , 54.  , 80.  ])             ],
      dtype=object)

In [ ]:
output = pd.DataFrame({
    'age': X_train['Age'],
    'age_trf':X_train_t[:,0],
    'fare':X_train['Fare'],
    'fare_t':X_train_t[:,1]
})

In [ ]:
output['age_labels']= pd.cut(x=X_train['Age'],
                                        bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels']= pd.cut(x=X_train['Fare'],
                                        bins=trf.named_transformers_['second'].bin_edges_[0].tolist())


In [ ]:
output.sample(5)

,age,age_trf,fare,fare_t,age_labels,fare_labels
499,24.0,5.0,7.7958,2.0,"(23.0, 25.0]","(7.775, 7.896]"
315,26.0,6.0,7.8542,2.0,"(25.0, 28.0]","(7.775, 7.896]"
623,21.0,4.0,7.8542,2.0,"(19.0, 21.0]","(7.775, 7.896]"
174,56.0,14.0,30.6958,10.0,"(54.0, 80.0]","(26.55, 31.275]"
621,42.0,12.0,52.5542,12.0,"(38.0, 42.0]","(51.479, 76.292]"


In [ ]:
clf = DecisionTreeClassifier() # Create an instance of DecisionTreeClassifier
clf.fit(X_train_t,y_train)
y_pred2 = clf.predict(X_test_t)

In [ ]:
accuracy_score(y_test,y_pred2)

0.6363636363636364

In [ ]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

np.float64(0.6345070422535211)

## Binarization

Convert Continous value to binary value